## Instalaciones

In [ ]:
%pip install datasets
%pip install transformers
%pip install transformers datasets torch

## Librerias

In [4]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

In [ ]:
COL = "OBSERVACION TERRENO"
MODEL = "dccuchile/bert-base-spanish-wwm-cased"  # BETO

# 1) Cargar datos
df = pd.read_excel("Consolidado_GPON_Final-1.xlsx", engine='openpyxl')
df = df[[COL]].dropna().rename(columns={COL:"text"})

# 2) Etiquetado débil (palabras clave simples)
buckets = {
    "Portafolio": [
        "cambio equipo","wifi","decos","canales","canales premium","cantidad","sport wins","hbo","megas",
        "velocidad","velocidad contratada","contraseña","validacion general","mac","co creada","portafolio de equipo"
    ],
    "Soporte": [
        "aprov","aprovionamiento","activar","aprovisionar","cam_eq","cambio equipo","cierre cbapon","cod_comp",
        "configurar vlan","contingencia","estabilizar pon","ip navegacion","puertos lan","reenvio registros",
        "validacion general","cambio ont","cambio modem","conexion en tr","reaprovisiona","reaprov",
        "aprovisionamiento","activar decos","instalar","instalacion", "equipo quemado"
    ],
    "Infraestructura": [
        "sin cobertura","red ocupada","nap trocado","actualizar infra","informacion infraestructura","infraestructura"
    ]
}
def suggest_label(text):
    t = str(text).lower()
    scores = {k:0 for k in buckets}
    for k, kws in buckets.items():
        for kw in kws:
            if kw in t:
                scores[k] += 1
    best = max(scores, key=scores.get)
    return best if scores[best] > 0 else "otros"

df["label_name"] = df["text"].apply(suggest_label)

# 3) Mapear etiquetas a ids
labels = sorted(df["label_name"].unique())
label2id = {name:i for i,name in enumerate(labels)}
id2label = {i:name for name,i in label2id.items()}
df["label"] = df["label_name"].map(label2id)

# 4) Split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])
train_ds = Dataset.from_pandas(train_df[["text","label"]].reset_index(drop=True))
test_ds  = Dataset.from_pandas(test_df[["text","label"]].reset_index(drop=True))

# 5) Tokenizador y modelo
tok = AutoTokenizer.from_pretrained(MODEL)
def tok_fn(batch):
    return tok(batch["text"], truncation=True, padding=True)

train_ds = train_ds.map(tok_fn, batched=True)
test_ds  = test_ds.map(tok_fn, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=len(labels), id2label=id2label, label2id=label2id)

# 6) Entrenamiento mínimo
args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_steps=50,
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01
)

trainer = Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=test_ds)
trainer.train()

# 7) Ejemplo de inferencia
pred = trainer.predict(test_ds.select(range(min(5, len(test_ds)))))
import numpy as np
y = pred.predictions.argmax(axis=1)
print("\nEtiquetas: ", id2label)
print("Predicciones de ejemplo:", y.tolist())


c:\Users\marin\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\marin\.cache\huggingface\hub\models--dccuchile--bert-base-spanish-wwm-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map: 100%|██████████| 26320/26320 [00:06<00:00, 4360.26 examples/s]
Xet Stor

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
